# Timing of oab
## Run these once, at the start of the notebook

In [ ]:
import logging
import oab

In [1]:
import pickle
import time
from collections import Counter
from time import perf_counter

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rich import print

In [2]:
# make results list
results_table = []

## Personalize the following cell

In [3]:
# personalize this for running the notebook in different ways
dataset_name = "mnist"
algo_name = "RF"

my_class = 7

how_many_images = 100  # how many images each combination of the above?

## Run these after each "personalization"
To get results on multiple datasets, algorithms, classes

### Cells *not* timed
Data loading and more

In [ ]:
(X_train, Y_train), (X_test, Y_test), (X_tree, Y_tree) = oab.get_data(dataset_name)
my_dom = oab.Domain(dataset_name, algo_name)
index = {}
points = {}
for my_class in range(10):
    index[my_class] = np.where(Y_test == my_class)[0]
my_dom.load()

### Timed cells

In [ ]:
start = perf_counter()

for i, my_index in enumerate(index[my_class]):
    testpoint = oab.TestPoint(X_test[my_index], my_dom)
    logging.info(f"start explanation of index {my_index}")
    exp = oab.Explainer(testpoint, howmany=10)

    print("for test", exp.testpoint.latent.a)
    plt.imshow(exp.testpoint.a.astype("uint8"), cmap="gray")
    plt.title(
        f"TestPoint - black box predicted class: {exp.testpoint.blackboxpd.predicted_class}"
    )
    plt.show()

    print("for treepoint", exp.target.latent.a)
    plt.imshow(exp.target.a.astype("uint8"), cmap="gray")
    plt.title(
        f"TreePoint - black box predicted class: {exp.target.blackboxpd.predicted_class}"
    )
    plt.show()

    print("counterrules")
    print(exp.target.latentdt.counterrules)

    print("for this target treepoint:")
    print(exp.target)
    print("# factuals")
    for point in exp.factuals:
        plt.imshow(point.a.astype("uint8"), cmap="gray")
        plt.title(
            f"factual - black box predicted class: {point.blackboxpd.predicted_class}"
        )
        plt.show()
    print("## factuals BUT closest, instead of furthest")
    for point in exp._factuals_default(closest=True):
        plt.imshow(point.a.astype("uint8"), cmap="gray")
        plt.title(
            f"factual - black box predicted class: {point.blackboxpd.predicted_class}"
        )
        plt.show()
    print("# counterfactuals")
    # the following is because sometimes an error TypeError: 'NoneType' object is not iterable
    if exp.counterfactuals:
        for point in exp.counterfactuals:
            print("miao1")
            plt.imshow(point.a.astype("uint8"), cmap="gray")
            print("miao2")
            plt.title(
                f"counterfactual - black box predicted class: {point.blackboxpd.predicted_class}"
            )
            print("miao3")
            plt.show()
    print("## New method for **more** counterfactuals!")
    for point in exp.more_counterfactuals():
        # this is generating right here the "more" counterfactuals
        plt.imshow(point.a.astype("uint8"), cmap="gray")
        plt.title(
            f"counterfactual - black box predicted class: {point.blackboxpd.predicted_class}"
        )
        plt.show()

    if i >= how_many_images:
        # how many points to do per class
        break


end = perf_counter()
# compute time
execution_time = end - start
# append into results_table
results_table.append(
    {
        "Dataset": dataset_name,
        "Algo": algo_name,
        "Class": my_class,
        "Time": execution_time,
    }
)

## Run this to show the results table

In [1]:
results_dataframe = pd.DataFrame.from_records(results_table)
results_dataframe

NameError: name 'pd' is not defined